# Cross-Section Demo: SPY / GLD / SLV

Using FRED macro factors + pandas-datareader prices (stooq) to trade SPY/GLD/SLV.

In [1]:
from config import settings
from data_fetch import DataFetcher
from factors import (
    build_deleveraging,
    build_funding_stress,
    build_market_liquidity,
    build_net_liquidity_flow,
    build_net_liquidity_level,
    build_reserves_rrp_rotation,
    combine_all_signals,
    combine_macro_chain,
    combine_net_liquidity,
)
from factor_tests import evaluate_factors
from strategy import build_position_matrix, run_cross_section_backtest, summarize

import plotly.graph_objects as go
import pandas as pd

asset_tickers = ["SPY", "GLD", "SLV"]

In [2]:
fetcher = DataFetcher()
print(f"Date range: {settings.start_date} -> {settings.end_date}")

# FRED factors (weekly)
buckets = fetcher.fetch_macro_chain_inputs(
    start=settings.start_date,
    end=settings.end_date,
    weekly_freq=settings.weekly_freq,
    include_sp500=True,
    use_cache=True,
)
print("Fetched weekly rows (FRED):", len(buckets))

# Asset prices (stooq via pandas-datareader)
asset_prices = fetcher.fetch_assets(
    tickers=asset_tickers,
    start=settings.start_date,
    end=settings.end_date,
    weekly_freq=settings.weekly_freq,
    source="stooq",
    use_cache=True,
)
print("Fetched weekly rows (assets):", len(asset_prices))

Date range: 2014-01-01 -> 2025-12-10
Fetched weekly rows (FRED): 624
Fetched weekly rows (assets): 623


In [3]:
# Fed plumbing factors
nl_level = build_net_liquidity_level(buckets, level_window=252)
nl_flow = build_net_liquidity_flow(buckets, k=21, flow_window=252)
nl_rot = build_reserves_rrp_rotation(buckets, k=21, flow_window=252)
net_liq = combine_net_liquidity(nl_level, nl_flow, nl_rot)

# Macro chain factors
funding = build_funding_stress(buckets, window=26)
mkt_liq = build_market_liquidity(buckets, window=26)
delev = None
if {"CFTC_ES_LONG", "CFTC_ES_SHORT"}.issubset(buckets.columns):
    delev = build_deleveraging(buckets, window=26)
macro_chain = combine_macro_chain(funding, mkt_liq, delev)

# Final blend
factors = combine_all_signals(net_liq, macro_chain)
factors.head()

,NL_level_raw,NL_level_z,NL_flow_21,NL_flow_z,NL_rot_21,NL_rot_z,LIQ_signal_raw,LIQ_signal,FUND_ted_z,FUND_cp_ff_z,FUND_cp_sofr_z,FUND_stress,LIQ_vix_z,LIQ_hy_oas_z,LIQ_ig_oas_z,LIQ_stress,MACRO_chain_raw,MACRO_chain,ALL_signal_raw,ALL_signal
2014-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-08,2333737.455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-15,2447814.248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-22,2444998.289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-29,2429766.724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Diagnostics on SP500 (optional sanity check)
cols = [
    "NL_level_z",
    "NL_flow_z",
    "NL_rot_z",
    "LIQ_signal",
    "FUND_stress",
    "LIQ_stress",
    "MACRO_chain",
    "ALL_signal",
]
if "DELEV_stress" in factors.columns:
    cols.insert(6, "DELEV_stress")

diag = evaluate_factors(
    factors[cols],
    price=buckets["SP500"],
    horizon=1,
    signal_lag=1,
)
diag

,n_obs,horizon,signal_lag,ic_pearson,ic_spearman,ic_tstat,mean_ret,vol_ret,sharpe_ann,hit_rate,turnover,decay_lag1,decay_lag4,decay_lag12
name,,,,,,,,,,,,,,
NL_level_z,370,1,1,0.019793,-0.016154,0.379764,0.000481,0.021469,0.161494,0.497297,0.098124,0.990870,0.942186,0.784893
NL_flow_z,349,1,1,0.099610,0.044530,1.864802,0.002433,0.029298,0.598776,0.541547,0.185426,0.970998,0.822172,0.378336
NL_rot_z,349,1,1,0.113037,0.084101,2.119234,0.002761,0.035062,0.567793,0.538682,0.153728,0.980014,0.872793,0.545280
LIQ_signal,349,1,1,0.065865,0.048071,1.229608,0.001609,0.027015,0.429394,0.541547,0.206940,0.957706,0.788400,0.355495
FUND_stress,375,1,1,0.079744,0.039587,1.545034,0.001956,0.026393,0.534537,0.493333,0.416599,0.773924,0.442669,-0.020259
LIQ_stress,521,1,1,0.020995,0.082700,0.478402,0.000464,0.029458,0.113631,0.495202,0.401784,0.800869,0.487460,0.045173
MACRO_chain,375,1,1,0.063198,0.106867,1.223001,0.001551,0.027743,0.403021,0.506667,0.422661,0.820989,0.487609,0.006370
ALL_signal,349,1,1,0.080752,0.073676,1.509177,0.001972,0.027987,0.508156,0.555874,0.247489,0.942169,0.737734,0.256404


In [5]:
# Cross-section backtest: equal-weight SPY/GLD/SLV when liquidity signal > 0
signal = factors["ALL_signal"]
weights = pd.DataFrame(index=signal.index, columns=asset_tickers)
weights[:] = signal.values[:, None]
weights = weights.clip(lower=0.0)
weights = weights.div(weights.sum(axis=1).replace(0, 1), axis=0)
weights = weights.shift(1).fillna(0.0)  # lag to avoid look-ahead

result = run_cross_section_backtest(asset_prices, weights, trading_cost_bps=10.0)
stats = summarize(
    result.rename(columns={"port_ret_net": "strategy_ret_net", "cum_strategy": "cum_strategy"})
)
stats

C:\Users\Administrator\AppData\Local\Temp\ipykernel_31408\1968745362.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  weights = weights.div(weights.sum(axis=1).replace(0, 1), axis=0)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_31408\1968745362.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  weights = weights.shift(1).fillna(0.0)  # lag to avoid look-ahead


PerformanceStats(total_return=0.6761760798371261, annual_return=0.0440548340357414, annual_vol=0.09180537767277473, sharpe=0.47987204184015947, max_drawdown=-0.24050477473507448, n_periods=623)

In [6]:
# Plot equity curve vs equal-weight buy & hold
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=result.index, y=result["cum_strategy"], mode="lines", name="Strategy")
)
fig.add_trace(
    go.Scatter(x=result.index, y=result["cum_buyhold"], mode="lines", name="Buy & Hold (EW)")
)
fig.update_layout(
    title="SPY/GLD/SLV Liquidity Strategy vs Equal-Weight Buy & Hold",
    yaxis_title="Growth of $1",
    xaxis_title="Date",
    template="plotly_white",
    legend=dict(x=0.02, y=0.98),
)
fig.show()